# Проект: анализ базы данных сервиса для чтения книг 

## Оглавление:

1. [Описание проекта](#describe)
2. [Анализ данных](#research)
3. [Задание 1](#task_1)
4. [Задание 2](#task_2)
5. [Задание 3](#task_3)
6. [Задание 4](#task_4)
7. [Задание 5](#task_5)
8. [Выводы](#conclusion)

<a id="describe"></a> 
## Описание проекта

**Цель исследования:** 
необходимо проанализировать базу данных книг, чтобы сформулировать ценностное предложение для нового сервиса чтения книг. 


 **Задача проекта:**
* определить количество книг, вышедших после 1 января 2000 года;
* изучить количество обзоров и среднюю оценку книг;
* определить количество книг, у которых более 50 страниц;
* определить автора с самой высокой средней оценкой книг;
* определить среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Дата проведения исследования:** 21-22.03.2023

#### Структура данных: 


**Таблица books**


* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.


**Таблица authors**
* author_id — идентификатор автора;
* author — имя автора.


**Таблица publishers**
* publisher_id — идентификатор издательства;
* publisher — название издательства;


**Таблица ratings**
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.


**Таблица reviews**
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

#### Схема данных: 

![Схема данных](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

<a id="research"></a>
## Анализ данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# напишем функцию для вывода 
def sql_translation(query):
    return pd.io.sql.read_sql(query, con = engine)

In [3]:
books = '''
SELECT *  FROM books
LIMIT(5)
'''
books = sql_translation(books)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
book_count = '''
--Выведем количество книг
SELECT COUNT(DISTINCT book_id) 
FROM books
'''
book_count = sql_translation(book_count)
book_count

,count
0,1000


In [5]:
# выведем строки таблицы authors
authors = '''
SELECT *  FROM authors
LIMIT(5)
'''
authors = sql_translation(authors)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
authors_count = '''
--Выведем количество авторов
SELECT COUNT(DISTINCT author_id) 
FROM authors
'''
authors_count = sql_translation(authors_count)
authors_count

,count
0,636


In [7]:
# выведем строки таблицы publishers
publishers = '''
SELECT *  FROM publishers
LIMIT(5)
'''
publishers = sql_translation(publishers)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
publishers_count = '''
--Выведем количество издательств
SELECT COUNT(DISTINCT publisher_id) 
FROM publishers
'''
publishers_count = sql_translation(publishers_count)
publishers_count

,count
0,340


In [9]:
# выведем строки таблицы ratings
ratings = '''
SELECT *  FROM ratings
LIMIT(5)
'''
ratings = sql_translation(ratings)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
# выведем строки таблицы reviews
reviews = '''
SELECT *  FROM reviews
LIMIT(5)
'''
reviews = sql_translation(reviews)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


* на данном этапе были выгружены все таблицы и изучены. В библиотеке у нас находится 1000 книг 636 авторов, опубликованных 340 издательствами.

<a id="task_1"> </a>
## Задание 1: определить количество книг, выпущенных после 1 января 2020 года

In [12]:
book_count = '''
SELECT COUNT(book_id) AS book_count
FROM books
WHERE publication_date >= '2000-01-02'
'''
sql_translation(book_count)

,book_count
0,819


<a id="task_2"> </a>
## Задание 2: количество обзоров и среднюю оценку на каждую книгу

In [14]:
avg_book = '''
SELECT b.title AS title,
       COUNT(DISTINCT rw.review_id) AS count_review, 
       ROUND(AVG(r.rating), 2) AS avg_rating
FROM reviews AS rw 
RIGHT JOIN books AS b ON rw.book_id=b.book_id
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC, count_review DESC
'''
sql_translation(avg_book)

,title,count_review,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


* Самыми популярными книгами являются A Dirty Job (Grim Reaper #1) автора Christopher Moore, School's Out—Forever (Maximum Ride #2) - James Patterson, Moneyball: The Art of Winning an Unfair Game	- Michael Lewis. Книги, имеющие высокий рейтинг, являются фантастикой.

In [15]:
# количество уникальных отзывов
sql_translation(avg_book)['count_review'].sum()

2793

In [16]:
# посмотрим на количество уникальных отзывов
count_review = '''
SELECT COUNT(DISTINCT(review_id))
FROM reviews'''
sql_translation(count_review)

,count
0,2793


<a id="task_3"> </a>
## Задание 3: издательство, которое выпустило наибольшее число книг

In [17]:
top_publisher = '''
SELECT p.publisher AS publisher,
    COUNT(b.book_id) AS count_book
FROM publishers AS p
JOIN books AS b ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY count_book DESC
LIMIT(1)
'''
sql_translation(top_publisher)

,publisher,count_book
0,Penguin Books,42


* Издательство Penguin Books больше всех выпустило книг, их количество равно 42

<a id="task_4"> </a>
## Задание 4: автор с самой высокой средней оценкой книг

In [20]:
var_2_top_authur = ''' 
SELECT 
    t.author,
    AVG(t.avg_rating) AS avg_rating
FROM
    (SELECT a.author AS author,
            AVG(r.rating) AS avg_rating
    FROM books AS b
    LEFT JOIN authors AS a ON b.author_id = a.author_id
    LEFT JOIN ratings AS r ON b.book_id = r.book_id
    GROUP BY b.book_id,
             a.author_id
    HAVING COUNT(r.rating_id) >50 
    ) AS t
GROUP BY t.author
ORDER BY avg_rating DESC
LIMIT 5 '''

sql_translation(var_2_top_authur)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


* Самой популярной писательницей является Джоан Роулинг, которая написала всем известную серию книг "Гарри Поттер". Также в топ-5 входит Джон Рональд Руэл Толкин, который известен своими книгами «Хоббит, или Туда и обратно», «Властелин колец» и «Сильмариллион». На втором месте находится Маркус Зузак.

<a id="task_5"> </a>
## Задание 5: среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [21]:
avg_review = '''
SELECT ROUND(AVG(cnt_review),2)
FROM (SELECT username,
        COUNT(review_id) AS cnt_review
        FROM reviews AS r
        WHERE username IN (SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 50)
        GROUP BY username) AS cnt_review
'''
sql_translation(avg_review)

,round
0,24.33


<a id="conclusion"></a>
 ## Выводы

При выборе библиотеки, которая будет находится на сервисе, стоит обратить внимание на предпочтение пользоваталей. Самыми популярными жанрами являются фантастика, фэнтези и детективы. Самые популярные авторы это Джоан Роулинг, Джон Рональд Руэл Толкин, Маркус Зузак. В среднем пользователи, которые поставили больше 50 оценок, оставляют 24 обзора на книги.